# **Analisis Sentimen Twitter**
### Caesar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 26.7 MB/s 


In [ ]:
# !pip install torchtext==0.10.0

## 1. Preparation

### 1.1. Import Library

In [ ]:
import csv
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import ast
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from nltk.corpus import stopwords

### 1.2. Get Data and Data Cleaning

In [ ]:
# Ambil Data

# df1 = pd.read_csv("Aug (1).csv" , skiprows=8)
# df2 = pd.read_csv("Aug (2).csv" , skiprows=8)
# df3 = pd.read_csv("Aug (3).csv" , skiprows=8)
# df4 = pd.read_csv("Aug (4).csv" , skiprows=8)
# df5 = pd.read_csv("Aug (5).csv" , skiprows=8)
# df6 = pd.read_csv("Aug (6).csv" , skiprows=8)
# df7 = pd.read_csv("Aug (7).csv" , skiprows=8)
# df8 = pd.read_csv("Aug (8).csv" , skiprows=8)
# df9 = pd.read_csv("Aug (9).csv" , skiprows=8)
# df10 = pd.read_csv("Aug (10).csv" , skiprows=8)
# df11 = pd.read_csv("Aug (11).csv" , skiprows=8)
# frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

# df1 = pd.read_csv("Sep (1).csv" , skiprows=8)
# df2 = pd.read_csv("Sep (2).csv" , skiprows=8)
# df3 = pd.read_csv("Sep (3).csv" , skiprows=8)
# df4 = pd.read_csv("Sep (4).csv" , skiprows=8)
# df5 = pd.read_csv("Sep (5).csv" , skiprows=8)
# df6 = pd.read_csv("Sep (6).csv" , skiprows=8)
# df7 = pd.read_csv("Sep (7).csv" , skiprows=8)
# df8 = pd.read_csv("Sep (8).csv" , skiprows=8)
# frames = [df1, df2, df3, df4, df5, df6, df7, df8]

# df1 = pd.read_csv("Okt (1).csv" , skiprows=8)
# df2 = pd.read_csv("Okt (2).csv" , skiprows=8)
# df3 = pd.read_csv("Okt (3).csv" , skiprows=8)
# df4 = pd.read_csv("Okt (4).csv" , skiprows=8)
# df5 = pd.read_csv("Okt (5).csv" , skiprows=8)
# frames = [df1, df2, df3, df4, df5]

# df1 = pd.read_csv("Nov (1).csv" , skiprows=8)
# df2 = pd.read_csv("Nov (2).csv" , skiprows=8)
# frames = [df1, df2]

# df = pd.concat(frames)

# df.rename(columns = {'Title':'text'}, inplace = True)
# df['label_target'] = np.random.choice([0, 1], df.shape[0]) #masih random
# df = df[['text', 'label_target']]

In [ ]:
df = pd.read_csv("Oktober2.csv")

In [ ]:
df.info

<bound method DataFrame.info of      Unnamed: 0                                               text  \
0          2486  #TKO pelan pelan runtuh... Selanjutnya apa yan...   
1          1626  @0xminryunggg dulu tuh setauku tokocrypto admi...   
2          1723  @1st_pickOctober Udah sampai kak cuman di toko...   
3          2063  @bravepromote Aku pake indodax klo di tokocryp...   
4           109  @chaposynthetic Di binance link up dan down mu...   
..          ...                                                ...   
770         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
771         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
772        2582  Top 10 $high 1-week Influencers \n\nðŸ¥‡ bsc_d...   
773        1383    Wen $BONE @Tokocrypto ? https://t.co/boz779pYnC   
774        1281            Wen Proof of Tokocrypto TKO 2.0 ?\nðŸ¤£   

     label_target  343  267  
0               0  NaN  NaN  
1               0  NaN  NaN  
2               0  NaN  NaN  
3      

In [ ]:
# Membersihkan data
df = df.loc[df['text'].str.contains('okocrypto') == True]

searchfor = ['BestAsk Exchange', '@.*@.*', '#.*#.*', '^RT ', 'you ', 'Why',\
             'How', 'how', 'who', 'here', 'Market.*$.*', 'Hits.*$.*', ' be '\
             'Price.*$.*', 'Volume.*$.*', 'before', 'link:', 'Link:',\
             'Click', 'Hai Kak', 'okocrypto https:', '#FiturTerbaruTokocryptor',\
             '#Tokocrypto4nniversary', '#SiapLebihCuan', 'recent', ' is ',\
             'really', 'during', ' and ', ' this ', ' in ', ' to ', ' at ',\
             'Hai kak', 'Halo Kak', ' will ', 'ek dm', 'ek DM', ' with ',\
             ' my ', 'after', ' a ', ' it ', \
             r'[\u263a-\U0001f645].*[\u263a-\U0001f645]']

df = df.loc[df['text'].str.contains('|'.join(searchfor)) == False]

df["text"] = df["text"].apply(lambda x: x.replace("$", ""))
df["text"] = df["text"].apply(lambda x: x.replace(" @Tokocrypto", " Tokocrypto"))

df = df[df['text'].str.len()>30]
 
df.info

<bound method DataFrame.info of      Unnamed: 0                                               text  \
0          2486  #TKO pelan pelan runtuh... Selanjutnya apa yan...   
1          1626  @0xminryunggg dulu tuh setauku tokocrypto admi...   
2          1723  @1st_pickOctober Udah sampai kak cuman di toko...   
3          2063  @bravepromote Aku pake indodax klo di tokocryp...   
4           109  @chaposynthetic Di binance link up dan down mu...   
..          ...                                                ...   
770         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
771         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
772        2582  Top 10 high 1-week Influencers \n\nðŸ¥‡ bsc_da...   
773        1383      Wen BONE Tokocrypto ? https://t.co/boz779pYnC   
774        1281            Wen Proof of Tokocrypto TKO 2.0 ?\nðŸ¤£   

     label_target  343  267  
0               0  NaN  NaN  
1               0  NaN  NaN  
2               0  NaN  NaN  
3      

In [ ]:
# Cek duplikasi
print("tweet duplicated all column",df.duplicated().sum())
print("tweet duplicated by text",df.duplicated(subset=['text']).sum())
print("total",df.duplicated().sum() + df.duplicated(subset=['text']).sum())

tweet duplicated all column 0
tweet duplicated by text 1
total 1


In [ ]:
#Menghilangkan duplikasi tweet
print("before",df.shape)
df = df.drop_duplicates()
df = df.drop_duplicates('text', keep='last')
print("after",df.shape)

before (712, 5)
after (711, 5)


In [ ]:
#Menghapus Duplikat berdasarkan 7 Kata awal yang sama. Code ini untuk menghapus spam yang tidak terdeteksi.

idx = []
for i in range(df.shape[0]):
    #print(i)
    if i == (df.shape[0] - 1):
        break
    temp_a = df.iloc[i].text.split()[:7]
    temp_b = df.iloc[i+1].text.split()[:7]
    if temp_a == temp_b:
        idx.append(i+1)

df.info

<bound method DataFrame.info of      Unnamed: 0                                               text  \
0          2486  #TKO pelan pelan runtuh... Selanjutnya apa yan...   
1          1626  @0xminryunggg dulu tuh setauku tokocrypto admi...   
2          1723  @1st_pickOctober Udah sampai kak cuman di toko...   
3          2063  @bravepromote Aku pake indodax klo di tokocryp...   
4           109  @chaposynthetic Di binance link up dan down mu...   
..          ...                                                ...   
770         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
771         946  Tokocrypto: Perempuan Masih Ragu Investasi Kri...   
772        2582  Top 10 high 1-week Influencers \n\nðŸ¥‡ bsc_da...   
773        1383      Wen BONE Tokocrypto ? https://t.co/boz779pYnC   
774        1281            Wen Proof of Tokocrypto TKO 2.0 ?\nðŸ¤£   

     label_target  343  267  
0               0  NaN  NaN  
1               0  NaN  NaN  
2               0  NaN  NaN  
3      

In [ ]:
df.text.to_list()

['#TKO pelan pelan runtuh... Selanjutnya apa yang akan dilakukan Tokocrypto nih',
 '@0xminryunggg dulu tuh setauku tokocrypto adminya 5k deh.. \nDulu cv dibantu mba rita. Baik banget ngga ambil fee. Padahal 2x',
 '@1st_pickOctober Udah sampai kak cuman di tokocrypto ku yg bermasalah sengan pin google nya',
 '@bravepromote Aku pake indodax klo di tokocrypto fee nya geda ga??',
 '@chaposynthetic Di binance link up dan down muncul, kalau di Tokocrypto ga muncul mungkin karena error seperti minggu lalu, minggu lalu semua token etf di tokocrypto sempat ga muncul tapi akhirnya pulih lagi.\n\nSaran saya kalau mau trading token up dan down gitu fokus ke BTC & ETH up down aja.',
 '@cutefreulein Btw, tokocrypto skrg ga bs di ss ya? Huhuhu. Jd jelek buram deh tu gegara foto pake hp lain',
 '@GAHunterFess Kirim ke tokocrypto aja ntar cv disitu, tpi kayanya lumayan feenya',
 '@heroctaviano Tokocrypto w emailnya kena hack',
 '@hoteliercrypto Iya padahal ZEC dan XMR bagus ya kenapa di delist dari tok

In [ ]:
df['label_target'].value_counts()

0    327
1    255
2    129
Name: label_target, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711 entries, 0 to 774
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    711 non-null    int64  
 1   text          711 non-null    object 
 2   label_target  711 non-null    int64  
 3   343           0 non-null      float64
 4   267           0 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 33.3+ KB


In [ ]:
df.to_csv("dataset_skripsi.csv")